In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install youtokentome

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

import youtokentome as yttm
import numpy as np

import random
import math
import time
import pickle
from tqdm.auto import tqdm

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [6]:
# token_model = yttm.BPE(model='try_poems_embed_yttm.model', n_threads=-1)
token_model = yttm.BPE(model='/content/drive/MyDrive/Colab Notebooks/NLP_poems/voc10k_poems_embed_yttm.model', n_threads=-1)

In [7]:
class newDataset(Dataset):
    def __init__(self, data_list_of_list):
        self.data = data_list_of_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def cut_length(self):
#         middle_len = sum([len(text) for text in self.data])/len(self.data)
#         limit = int(middle_len)*2
        limit = 10
        self.data = [text if len(text)<limit else text[:limit] for text in self.data]

    def distribute_data(self, share_tr, share_val):  # examle = (0.8, 0.1)
        rand_i = np.random.permutation(len(self.data))
        n1 = int(len(self.data) * share_tr)
        n2 = int(len(self.data) * share_val)
        # return self.data[rand_i[0: n1]], self.data[rand_i[n1: n1 + n2]], self.data[rand_i[n1 + n2:]]
        return [self.data[i] for i in rand_i[0: n1]], \
               [self.data[i] for i in rand_i[n1: n1 + n2]], \
               [self.data[i] for i in rand_i[n1 + n2:]]
    
def padding(batch):
    pad_id = token_model.subword_to_id('<PAD>')
    
    batch = [torch.tensor(x) for x in batch]
    batch = pad_sequence(batch, batch_first=False, padding_value=pad_id)  # batch_first=True -> [batch size, len]
    return batch
    # return batch.to(device)

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers) # dropout = dropout
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):      
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
        #outputs = [src len, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        #hidden - it will be first hid states in decoder
        return outputs, hidden

In [9]:
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, hid dim]
        #encoder_outputs = [src len, batch size, hid dim]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, hid dim]
        #encoder_outputs = [batch size, src len, hid dim]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        #energy = [batch size, src len, hid dim]

        attention = self.v(energy).squeeze(2)
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [10]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout, attention):
        super().__init__()
        
        self.output_dim = output_dim  # the size of the vocabulary for the output/target.
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim) # 1 - size of dict emb, 2 - size of emb vec
        
        self.rnn = nn.GRU(hid_dim + emb_dim, hid_dim, n_layers) # dropout = dropout
        
        self.fc_out = nn.Linear(hid_dim * 2 + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):

        #input = [batch size]
        #hidden = [n layers, batch size, hid dim]
        #encoder_outputs = [src len, batch size, hid dim]
        
        input = input.unsqueeze(0) 
        #input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
                
        last_hidden = hidden[-1]
        #hidden = [batch size, hid dim]        
        a = self.attention(last_hidden, encoder_outputs)    
        #a = [batch size, src len]
        a = a.unsqueeze(1)
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #encoder_outputs = [batch size, src len, hid dim]
        
        weighted = torch.bmm(a, encoder_outputs)    
        #weighted = [batch size, 1, hid dim]
        weighted = weighted.permute(1, 0, 2)
        #weighted = [1, batch size, enc hid dim]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        #rnn_input = [1, batch size, hid dim + emb dim]
        
        output, hidden = self.rnn(rnn_input, hidden)
        #output = [1, batch size, hid dim], seq len will always be 1 in the decoder
        #hidden = [n layers, batch size, hid dim]
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.2):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio=0.75 -> use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence
        #hidden is the final hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        
        #first input to the decoder is the <bos> tokens
        input = trg[0,:]  
        
        for t in range(1, trg_len):
            
            #insert input token emb-g, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token, else use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [12]:
INPUT_DIM = token_model.vocab_size() 
OUTPUT_DIM = INPUT_DIM
ENC_EMB_DIM = 128 #128
DEC_EMB_DIM = 128 #128
HID_DIM = 256 #256
N_LAYERS = 2 
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

attn = Attention(HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [13]:
PAD_ID = token_model.subword_to_id('<PAD>')
criterion = nn.CrossEntropyLoss(ignore_index = PAD_ID)

In [14]:
def test(model, loader, criterion):
    
    model.eval()
    
    epoch_loss = 0
    outputs_list = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(tqdm(loader)):
            batch = batch.to(device, non_blocking=True)
            src = batch
            trg = batch

            output = model(src, trg, 0) #turn off teacher forcing
            outputs_list.append(torch.transpose(output, 0, 1))

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
      
    return epoch_loss / len(loader), outputs_list

In [15]:
# optimizer.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/NLP_poems/b7e10dr03_optim.pt'))
# print(optimizer.param_groups[0]['lr'])
# print(optimizer.param_groups[0]['initial_lr'])
# print(optimizer.param_groups)

In [16]:
# checkpoint = torch.load('trylr005_17k_checkpoint.pt')
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/NLP_poems/AdamW_one_C005_same_drEm2_voc10k_tf2_20406080len100_Att_checkp_best.pt', 
                        map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [17]:
# with open('data/try_test_data_token.pickle', 'rb', ) as file:
with open('/content/drive/MyDrive/Colab Notebooks/NLP_poems/data/voc10klen100_test_data_token.pickle', 'rb', ) as file:
    test_data_token = pickle.load(file)
test_loader_load = DataLoader(
    test_data_token, batch_size=32, shuffle=False, num_workers=1, pin_memory=True, collate_fn=padding)

In [18]:
test_loss, test_poems_embed = test(model, test_loader_load, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

  0%|          | 0/56 [00:00<?, ?it/s]

| Test Loss: 0.705 | Test PPL:   2.025 |


#### Get poems from test output and compare with orig test poems

In [19]:
generated_poems_token = []
generated_poems_len = []
PAD_ID = token_model.subword_to_id('<PAD>')
BOS_ID = token_model.subword_to_id('<BOS>')
EOS_ID = token_model.subword_to_id('<EOS>')
for batch in test_poems_embed:
    for text_word_probab in batch:
        text_vocab_ids = [torch.argmax(word).item() for word in text_word_probab]
        generated_poems_token.append(text_vocab_ids)
        generated_poems_len.append(len(text_vocab_ids))

generated_poems = token_model.decode(generated_poems_token, ignore_ids=[PAD_ID, EOS_ID]) 

orig_poems = token_model.decode(test_data_token, ignore_ids=[PAD_ID, BOS_ID, EOS_ID]) 

In [22]:
for id in range(20, 30):
  # print(generated_poems_len[id])
  # print(len(orig_poems[id].split(' ')))
  print(generated_poems[id])
  print(orig_poems[id])
  print()

с латинского скорее челюстью своей поднимет солнце муравей скорей вода с огнем смесится к нт в ов скорее кровь в бальзам целебный обратится чем наша кончится любовь . быть может самый рим прейдет быть может т т нам вернет невозвратимого марона может может там средь над над крепкой высью пелиона и и не было богов . все допустимости во всем и иительнымительнымительным быть может быть может у может у может
с латинского скорее челюстью своей поднимет солнце муравей скорей вода с огнем смесится к нт в ов скорее кровь в бальзам целебный обратится чем наша кончится любовь . быть может самый рим прейдет быть может т т нам вернет невозвратимого марона быть может там средь облаков над крепкой высью пелиона и нет и не было богов . все допустимо и во всем злым и властительным умом пора быть может усомниться

все были сказаны давно заветы сладостной свободы и прежде претворялись воды в животворящее вино . припомни брак еврейский в кане и чудо первое христа и омочи свои уста водою налитой в стакане 

#### Use metric to evaluate results

In [ ]:
embedding = nn.Embedding(OUTPUT_DIM, DEC_EMB_DIM) # 1 - size of dict emb, 2 - size of emb vec

embed_source_poems = embedding(torch.tensor(generated_poems_token)) # [poems num, word num, embed len]
embed_target_poems = embedding(torch.tensor(test_data_token))

ValueError: ignored

In [ ]:
total_score = 0
for i in range(len(embed_target_poems)):
    embed_source_poem = embed_source_poems[i]  # [word num, embed len]
    embed_target_poem = embed_target_poems[i]
    # print(embed_source_poem.shape , embed_target_poem.shape)
    # print(embed_source_poem, embed_target_poem)

    # [min, mean, max] = sentence emb-g
    v_s_min = torch.min(embed_source_poem)
    v_s_mean = torch.mean(embed_source_poem)
    v_s_max = torch.max(embed_source_poem)

    v_t_min = torch.min(embed_target_poem)
    v_t_mean = torch.mean(embed_target_poem)
    v_t_max = torch.max(embed_target_poem)

    v_s = torch.tensor([v_s_min, v_s_mean, v_s_max])
    v_t = torch.tensor([v_t_min, v_t_mean, v_t_max])
    # print(v_s, v_t)

    score = torch.matmul(v_s.t(), v_t) / (torch.linalg.norm(v_s, ord=2) * torch.linalg.norm(v_t, ord=2))
    # if source = orig, then score = 1.0, else score < 1.0
    total_score += (1.0 - score)
    
total_score = total_score.item()
print(total_score)

2.2617645263671875


In [ ]:
# print(token_model.vocab()[:30])
token_model.subword_to_id(' \n')
token_model.encode('где бы ты не был \n в каких бы краях . \n я бы всегда ')
token_model.encode('где бы ты не был \n в каких бы краях . \n я бы всегда ', output_type=yttm.OutputType.SUBWORD)

['▁где',
 '▁бы',
 '▁ты',
 '▁не',
 '▁был',
 '▁в',
 '▁каких',
 '▁бы',
 '▁краях',
 '▁.',
 '▁я',
 '▁бы',
 '▁всегда']

In [ ]:
a = torch.tensor(
    [
     [[1,2],[3,4],[5,6],[7,8]],
     [[0,0],[1,1],[2,2],[3,3]],
     [[-1,-1],[-2,-2],[-3,-3],[-4,-4]]
    ]
)
print(a.shape) # [3, 4, 2]
# в каждом стихе 3 слова, всего 4 стиха, каждое слово из 2х букв

a = torch.transpose(a, 0, 1)
print(a.shape) # [4, 3, 2]
print(a)